# **Preparation**

In [ ]:
import tensorflow as tf
import PIL
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Dropout, Dense, Softmax)
from tensorflow.keras.applications import mobilenet as _mobilenet
import random
import os
import numpy as np
import shutil

In [ ]:
conda install pytorch torchvision cuda80 -c soumith

In [ ]:
!pip install --upgrade moviepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 388 kB 5.0 MB/s 
     |████████████████████████████████| 26.9 MB 1.4 MB/s 
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110742 sha256=a09c6dc10d808b86b3fa140a2d16cce1eb5f222583c4c5cd2c676ad3f167715e
  Stored in directory: /root/.cache/pip/wheels/e4/a4/db/0368d3a04033da662e13926594b3a8cf1aa4ffeefe570cfac1
Successfully built moviepy
  Attempting uninstall: moviepy
    Found existing installation: moviepy 0.2.3.5
    Uninstalling moviepy-0.2.3.5:
      Successfully uninstalled moviepy-0.2.3.5


In [ ]:
import json
import os
from moviepy.editor import *
from moviepy import *

In [ ]:
! git clone https://github.com/kenshohara/video-classification-3d-cnn-pytorch.git

Cloning into 'video-classification-3d-cnn-pytorch'...
remote: Enumerating objects: 121, done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 121
Receiving objects: 100% (121/121), 158.63 KiB | 4.81 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
# Make sure you navigate to the github repo
%cd /content/video-classification-3d-cnn-pytorch

/content/video-classification-3d-cnn-pytorch


# **Defination**

从文件夹导出电影名字的目录

In [ ]:
# this function loads all films in the folder given as argument into the 'input' file (make sure you are in the right
# directory)

# 这个函数将作为参数的文件夹中的所有电影加载到 "输入 "文件中（确保你在正确的目录中）。

def loadVideosToInfile(folder):
    videos = os.listdir(folder)
    with open('input', 'w') as inputfile:
      for v in videos:
        if v.split('.')[-1] == 'mp4':
          inputfile.write(v+'\n')

把所有json文件输出成一个字典

In [ ]:
# This function should read all .json files in the output folder, and combine them into one large dictionary
# 这个函数应该读取输出文件夹中的所有.json文件，并将它们合并成一个大的字典。

def readAllOutput(outputFolder):
    # create an empty dictionary called 'dictionaries' to which you add the dictionary entries read 
    # from the output files
    dictionaries = {}
    # Read all output file names to a variable output_files (a list that contains all file names)
    output_files = os.listdir(outputFolder)
    print(output_files)
    # Start of the for loop to loop through all files, read them, and add their contents to 'dictionaries'
    for f in output_files:
        # first test if the file extension is a '.json file'
            if f.split('.')[-1] == 'json':
                file = open(os.path.join(outputFolder, f))
                segments = json.load(file)
            # if it is a json file, we open the file
            
            # Then load the file content to a variable called 'segments' with the json library

            # another for loop, to add the content of segments to dictionaries. As a key use the moviename.
                for s in segments:
                    dictionaries[s['video']] = s
    
    return dictionaries

#dicts = readAllOutput('../output')
#dicts.keys()

输出片段信息

In [ ]:
# here keyFilmName is the name of the film that you want to find a match for, and the feature dictionary is 
# the dictionary that is given as output by the readAllOutput function
# 这里 keyFilmName 是你想找到匹配的电影名称，而特征字典是dictionary，该dictionary由 readAllOutput 函数输出。


def searchForMatch(keyFilmName, featureDictionary, frame = None, fps = 25, nBestMatches = 1):
    # create variables to keep track of the best clips
    
    fragments = []
    
    # first read the features of the keyFilm into keyFeatures
    keyClips = featureDictionary[keyFilmName]['clips']
    keyFeatures = []
    for c in keyClips:
        keyFeatures.append(np.array(c['features']))
        
    if frame:
        ft = 16/fps
        
        sf = int(frame[0]/ft)
        ef = int(min(frame[1]/ft, len(keyFeatures)-1))
        
        keyFeatures = keyFeatures[sf:ef]
    
    # now enter a for loop to go through all films, and see what the best matching sequence is 
    # (we make sure not to check the keyFilm itself by checking it's not that filename)
    for film in featureDictionary.keys():
        if not film == keyFilmName:
            filmClips = featureDictionary[film]['clips']
            filmFeatures = []
            for c in filmClips:
                filmFeatures.append(np.array(c['features']))
                
            for i in range(len(filmFeatures)-len(keyFeatures)):
                distance = 0
                for j in range(len(keyFeatures)):
                    d = np.linalg.norm(filmFeatures[i+j]-keyFeatures[j])
                    distance += d
                    
                distance = distance/len(keyFeatures)
                
                a = fragmentsExists(fragments, [film, i, len(keyFeatures), distance])
                
                if a:
                    fragments = a
                elif len(fragments) < nBestMatches:
                    fragments.append([film, i, len(keyFeatures), distance])
                    fragments.sort(key = lambda x : x[-1])
                elif distance < fragments[-1][-1]:
                    fragments.pop()
                    fragments.append([film, i, len(keyFeatures), distance])
                    fragments.sort(key = lambda x : x[-1])
                    
                    
            # so now here 'film' is a filename (which is the key in the feature dictionary of that film)
            # first load the features in a 'filmFeatures' list like we did for key Features
            # then look at the provided code from the lecture to see how to iterate through the other
            # list of features to find the best segment. Now just make sure to also register what film
            # name the best matching segment is from. 
            
    # create a list containing the filename, start and nr of clips
    
    return fragments

    # First implement it for the best matching one, then expand it to return the n best matching ones as a list,
    # with n as an argument in the function

测试是否存在

In [ ]:
def fragmentsExists(fragments, nf):
    for f in fragments:
        r = range(nf[1], nf[1]+nf[2])
        if f[0] == nf[0] and (f[1] in r or f[1]+f[2] in r):
            if f[-1] > nf[-1]:
                fragments.remove(f)
                fragments.append(nf)
                fragments.sort(key = lambda x : x[-1])
            return fragments
    return False

保存的前置步骤

In [ ]:
# this is a helper function to which you give the fragment (the output of the searchForMatch function), and it returns
# a moviepy film object (which you can then save)
# 这是一个辅助函数，你把片段（searchForMatch函数的输出）给它，然后它返回一个moviepy电影对象（然后你可以保存它）


def extractFragment(fragment, filmFolder):
    # here adapt the code from the lecture, but make it so that it works for any input fragment list
    # the output of the function above: [bestRangeFilmName, bestRangeStart, len(keyFeatures)]
    
    filmFileName = fragment[0]
    start = fragment[1]
    nrClips = fragment[2]
    
    film = VideoFileClip(os.path.join(filmFolder, filmFileName))
    tsfps = film.fps
    tsFrameLength = 1/tsfps
    
    nrFrames = nrClips * 16
    startFrame = start * 16
    
    startTime = startFrame * tsFrameLength
    endTime = startTime + nrFrames * tsFrameLength
    
    filmFragment = film.subclip(startTime, endTime)
    
    
    return filmFragment

    # This function should read all .json files in the output folder, and combine them into one large dictionary

# **Run**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


输入电影文件夹

In [ ]:
input_root = '/content/drive/MyDrive/RC11_2022-23/videotoproress' # 输入的需要处理的电影在这里
processed_root = '/content/drive/MyDrive/RC11_2022-23/video_done' # 过程文件夹 处理好的会自己塞进去
output_root = '/content/drive/MyDrive/RC11_2022-23/output' # 不是最终导出文件 只是过程量


In [ ]:
# call function to load the filenames to 'input'
loadVideosToInfile(input_root)

导出json文件

In [ ]:
# 记得改video_root的路径和output的名字↓↓↓

In [ ]:
# run the model to process all films currently in the input_films folder, and save it to a json file, here named by the date
# 运行模型，处理当前input_films文件夹中的所有影片，并将其保存到一个json文件中，这里以日期命名。

!python main.py --input input --video_root /content/drive/MyDrive/RC11_2022-23/videotoproress --output /content/drive/MyDrive/RC11_2022-23/output/walking21222.json --model /content/drive/MyDrive/RC11_2022-23/pretrainedmodel/resnet-34-kinetics.pth --mode feature 

loading model /content/drive/MyDrive/RC11_2022-23/pretrainedmodel/resnet-34-kinetics.pth
/content/drive/MyDrive/RC11_2022-23/videotoproress/walkingsmall.mp4
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheo

将输入过的电影移到另一个文件夹

In [ ]:
# a small piece of code that moves all processed input films to the processed films folder
# 一小段代码，将所有处理过的输入影片移到处理过的影片文件夹中。

input_films_processed = os.listdir(input_root)
for file in input_films_processed:
    try:
        shutil.move(os.path.join(input_root,file), processed_root)
    except:
        print(file + ' not moved')

In [ ]:
# read all output with the function declared above
# 用上面声明的函数读取所有输出

dicts = readAllOutput(output_root)

['output2.json', 'output.json', 'outputfeature.json', 'VideosToProcess051-070.json', 'VideosToProcess026-030.json', 'VideosToProcess031-050.json', 'VideosToProcess011-015.json', 'VideosToProcess016-025.json', 'VideosToProcess001-010.json', 'onetotwentyoutput.json', 'Videoprocessed40to50.json', 'output395to390.json', 'trail.json', 'output350to60.json', 'output355to60.json', 'output340to30.json', 'output350to40clip.json', 'output330to20.json', 'output330to202.json', 'output310to07.json', 'output307to04.json', 'output304to00.json', 'special.json', '300to295.json', '295to292.json', '292to290.json', 'streemoremosaic.json', 'coffeiso.json', 'working12.json', 'walking21222.json']


找到最匹配的片段

In [ ]:
# look for the best matching sequence to the input file (needs to be in dicts for it to work)
# 寻找与输入文件最匹配的序列（需要在dicts中才能工作）。

keyName = 'walkingsmall.mp4'
s = 0
d = 12

fragment = searchForMatch(keyName, dicts, frame = [s, s+d], nBestMatches = 10)

In [ ]:
fragment

[['Beuys-Joseph_Ausfegen_1972.mp4', 3012, 18, 18.502649382380092],
 ['Brigates_Esmigranti_1997.mp4', 10, 18, 18.866350071663234],
 ['Blake-Jeremy_Guccinam_2000.mp4', 860, 18, 18.87085154561062],
 ['Ballard_Moving-Pictures_Chris-Petit-on-JG-Ballard.mp4',
  521,
  18,
  18.88975503616356],
 ['Blake-Jeremy_Guccinam_2000.mp4', 63, 18, 18.917325127548466],
 ['Assayas-Olivier_Sans_Titre_Man-Yuk_Maggie-Cheung_1997.mp4',
  306,
  18,
  18.93648857359819],
 ['Blake-Jeremy_Guccinam_2000.mp4', 156, 18, 18.960681004778863],
 ['Blake-Jeremy_Guccinam_2000.mp4', 955, 18, 18.973158804576215],
 ['Blake-Jeremy_Guccinam_2000.mp4', 189, 18, 18.981830490060272],
 ['Blake-Jeremy_Guccinam_2000.mp4', 988, 18, 19.053886829394727]]

提取片段

In [ ]:
# ↓↓↓改片段长

In [ ]:
# extract the fragment from the film
# 抽出影片中的片段

fragmentFilms = []
for f in fragment[1:5]:
    fragmentFilms.append(extractFragment(f, processed_root))

In [ ]:
fragmentFilms

保存片段

In [ ]:
# 最终导出文件路径，记得每次要改以防覆盖↓↓↓

In [ ]:
# save the film fragment
for i in range(len(fragmentFilms)):
  fragmentFilms[i].write_videofile("/content/drive/MyDrive/RC11_2022-23/VideoCUT/videocut" + str(i) + ".mp4", audio_codec='aac')

Moviepy - Building video /content/drive/MyDrive/RC11_2022-23/VideoCUT/videocut0.mp4.
MoviePy - Writing audio in videocut0TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/RC11_2022-23/VideoCUT/videocut0.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/RC11_2022-23/VideoCUT/videocut0.mp4
Moviepy - Building video /content/drive/MyDrive/RC11_2022-23/VideoCUT/videocut1.mp4.
MoviePy - Writing audio in videocut1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/RC11_2022-23/VideoCUT/videocut1.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/RC11_2022-23/VideoCUT/videocut1.mp4
Moviepy - Building video /content/drive/MyDrive/RC11_2022-23/VideoCUT/videocut2.mp4.
MoviePy - Writing audio in videocut2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/RC11_2022-23/VideoCUT/videocut2.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/RC11_2022-23/VideoCUT/videocut2.mp4
Moviepy - Building video /content/drive/MyDrive/RC11_2022-23/VideoCUT/videocut3.mp4.
MoviePy - Writing audio in videocut3TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/RC11_2022-23/VideoCUT/videocut3.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/RC11_2022-23/VideoCUT/videocut3.mp4
